# Exercise 2: Sequential Patterns

This execise provides an introduction to how annotations are combined in sequential patterns.

First, we define some input text for the following examples.

In [ ]:
%%documentText
The dog barked at the cat.
Dogs, cats and mice are mammals.
Zander and tuna are fishes.
This text was created approx. on 13.04.2021.

Then, we provide some initial Animal annotations using an dictionary.

In [ ]:
DECLARE Animal;
WORDLIST AnimalList = 'animals.txt';
MARKFAST(Animal, AnimalList, true);

Now, we declare a new annotation type "AnimalEnum" and annotate enumerations of animals using composed rule elements and a quantifier ("+" or "*").

In [ ]:
DECLARE AnimalEnum;
(Animal (COMMA Animal)* ANY+{OR(IS(COMMA),REGEXP("and|or"))} Animal){-> AnimalEnum};

//COLOR(Animal, "lightgreen");
COLOR(AnimalEnum, "pink");

Lets reset the document (CAS) and try something different.

In [ ]:
%resetCas

In [ ]:
%%documentText
The dog barked at the cat.
Dogs, cats and mice are mammals.
Zander and tuna are fishes.
This text was created approx. on 13.04.2021.

 We declare a new annotation type "Sentence" and create a Sentence annotation for each sentence in the input document.

In [ ]:
// Lets also switch to a different output display mode
// we list the detected sentences in a table
%displayMode CSV
%csvConfig Sentence
DECLARE Sentence;

// This rule works in this exmaple, but can cuase problems in larger document
// Thus is should not be used:
//((# PERIOD){-PARTOF(Sentence)-> Sentence})+;

// we use two rules instead:
(# PERIOD){-> Sentence};
PERIOD (# PERIOD){-> Sentence};

This initial try to annotate sentence has several problems. Let's introduce a "helper" annotation for sentence ends and improve the resulting annotations.

In [ ]:
// remove all sentences
s:Sentence{-> UNMARK(s)};

DECLARE SentenceEnd;
// a period is a "SentenceEnd" if it is not followed by a number or a small writtern word.
// the optional "_" is a special matching condition. It is also fulfilled if nothing is left to match.
PERIOD{-> SentenceEnd} _{-PARTOF(NUM), -PARTOF(SW)};

// two simple rules are somtimes better (clearer/faster) than one complex rule.
(# SentenceEnd){-> Sentence};
SentenceEnd (# SentenceEnd){-> Sentence};

In the next cell, we declare four new annotation types: "Day", "Month", "Year" and "Date".
We create a single rule for detecting dates of the form "DD.MM.YYYY". 
This single rule should create four annotations:
A "Date" annotation for the complete date mention.
A "Day" annotation for the two digits of the day.
A "Month" annotation for the two digits of the month.
A "Year" annotation for the four digits of the year.

In [ ]:
DECLARE Date, Day, Month, Year;

//we restrict the number using a regex
(NUM{REGEXP("..")-> Day} PERIOD 
    NUM{REGEXP("..")-> Month} PERIOD 
    NUM{REGEXP(".{4}")-> Year}){-> Date};

COLOR(Day, "pink");
COLOR(Month, "lightgreen");
COLOR(Year, "lightblue");
COLOR(Date, "lightgrey");

It is often very useful to specify more than one action in a rule. This way, we can detect multiple entities using a single sequential pattern and only in combination with other information.